## Unveiling Urban Road Challenges - NYC Pothole Prediction

In this Notebook, we are performing Data Cleaning and Feature Engineering.

In [347]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import re
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

#### Load and Describe Data

In [348]:
df = pd.read_csv("../Data/Street_Pothole_Work_Orders_-_Closed__Dataset_.csv") #importing dataset
df.head(10)

,the_geom,DefNum,InitBy,HouseNum,OFT,OnFaceName,OnPrimName,FrmPrimNam,ToPrimName,SpecLoc,Boro,Source,RptDate,RptClosed,Shape_Leng
0,MULTILINESTRING ((-73.88643460204372 40.630776...,DB2004041154,YRD,NaN,373480377030377030,ROCKAWAY PY,ROCKAWAY PARKWAY,BELT PARKWAY,BELT PARKWAY,NaN,B,YRD,02/08/2004,01/27/2023,557.690465
1,MULTILINESTRING ((-73.94380842208851 40.584992...,DB2004287021,CSC,NaN,318630377030377030,BEDFORD AV,BEDFORD AVENUE,BELT PARKWAY,BELT PARKWAY,SB ON BEDFORD_UNDERNEATH OVERPASS,B,CTZ,10/13/2004,02/14/2013,109.124234
2,MULTILINESTRING ((-73.98570910180842 40.641185...,DB2006085009,CSC,NaN,306380308480308530,13 AV,13 AVENUE,38 STREET,39 STREET,I/F/O B16 BUS STOP,B,CTZ,03/26/2006,10/11/2014,260.495098
3,MULTILINESTRING ((-73.93487210852416 40.679593...,DB2007166005,CSC,1688,342730355530375430,FULTON ST,FULTON STREET,LEWIS AVENUE,SCHENECTADY AVENUE,POTHOLE NEAR CURB I/F/O ADDRESS,B,CTZ,06/15/2007,01/14/2011,549.255973
4,MULTILINESTRING ((-73.97793940872546 40.689806...,DB2009352004,CSC,45,380030331530342730,ST FELIX ST,ST FELIX STREET,DEKALB AVENUE,FULTON STREET,"4"" DEEP",B,CTZ,12/18/2009,01/06/2010,836.436669
5,MULTILINESTRING ((-73.97501506812618 40.640761...,DB2009362038,CSC,514,313830335030367730,AV C,AVENUE C,EAST 5 STREET,OCEAN PARKWAY,ONE POTHOLE I/F/O DRIVE WAY RIGHT OFF THE CURB,B,CTZ,12/28/2009,01/04/2010,385.456668
6,MULTILINESTRING ((-73.92232548215823 40.586294...,DB2009362066,CSC,NaN,342890356960375990,GAIN CT,GAIN COURT,LOIS AVENUE,SEBA AVENUE,CLOSER TO SEBA AVE,B,CTZ,12/28/2009,01/05/2010,529.261511
7,MULTILINESTRING ((-73.96802285162667 40.624450...,DB2009363041,CSC,808,314080335130335180,AV J,AVENUE J,EAST 8 STREET,EAST 9 STREET,IN FRONT OF ADDRESS NEAR BUS STOP,B,CTZ,12/29/2009,01/04/2010,259.706337
8,MULTILINESTRING ((-73.9080474319009 40.6954161...,DB2009363045,CSC,1402,347030352130394730,HANCOCK ST,HANCOCK STREET,IRVING AVENUE,WYCKOFF AVENUE,2 POTHOLES IN FRONT OF 1402 HANCOCK ST,B,CTZ,12/29/2009,01/10/2010,708.233651
9,MULTILINESTRING ((-73.95291388704382 40.706700...,DB2009364034,CSC,NaN,323230349430364630,BDWY,BROADWAY,HEWES STREET,NEW MONTROSE AVENUE,SEVERAL ON STREET,B,CTZ,12/30/2009,01/10/2010,304.584886


In [349]:
df.shape

(360730, 15)

In [350]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360730 entries, 0 to 360729
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   the_geom    360730 non-null  object 
 1   DefNum      360730 non-null  object 
 2   InitBy      360730 non-null  object 
 3   HouseNum    236000 non-null  object 
 4   OFT         360730 non-null  int64  
 5   OnFaceName  360067 non-null  object 
 6   OnPrimName  360653 non-null  object 
 7   FrmPrimNam  360700 non-null  object 
 8   ToPrimName  360624 non-null  object 
 9   SpecLoc     120304 non-null  object 
 10  Boro        360730 non-null  object 
 11  Source      360730 non-null  object 
 12  RptDate     360730 non-null  object 
 13  RptClosed   360730 non-null  object 
 14  Shape_Leng  360730 non-null  float64
dtypes: float64(1), int64(1), object(13)
memory usage: 41.3+ MB


We have 360730 rows and 15 columns in this data. Out of 15 Features, we have 13 objects, 1 Float and 1 Int.

### Data Cleaning

In [351]:
# checking nulls in the dataset
total_null_count = df.isnull().sum()
print(total_null_count)

the_geom           0
DefNum             0
InitBy             0
HouseNum      124730
OFT                0
OnFaceName       663
OnPrimName        77
FrmPrimNam        30
ToPrimName       106
SpecLoc       240426
Boro               0
Source             0
RptDate            0
RptClosed          0
Shape_Leng         0
dtype: int64


We have a large number of nulls in these columns "House_num", "specific", "OnFaceName", and "ToPrimName" locations. The first approach would be to fill in the missing data as we have a large number of nulls we can't actually replace or impute the missing values from other columns.
The approach we'll be taking is to feature engineer the required data from the "the_geom" column, we'll reverse the longitudes and latitudes to extract the address for every location.
I am dropping the columns "House_Num" and "SpecLoc" as we won't be needing them.

In [352]:
df = df.drop(['HouseNum', 'SpecLoc'], axis=1) 

The Data starts from the year 2004 whereas there is only 1 report per year for potholes between year 2004 and 2008. There are only 30 reports for the year 2009. So I am filtering the data starting from the year 2010 to remove the outliers and to have real-time insights

In [353]:
# convert datetime object to datetime data type
df['RptDate'] = df['RptDate'].astype('datetime64[ns]')  
df['RptClosed'] = df['RptClosed'].astype('datetime64[ns]')

date_threshold = pd.to_datetime('2009-12-31')

df = df[df['RptDate'] > date_threshold]

Resetting the index as it will be difficult to interpret the data without a numeric index, we already have "DefNum" but it is hard to interpret easily as it have string syntax

In [354]:
df['index'] = range(0, len(df))

### Feature Engineering
To improve our model and make it learn more about the pothole occurences I have feature engineered some columns.
* month (month of a pothole)
* days to fix a pothole
* zipcode (zip code of a pothole)
* street names (street name of a pothole)
* weather patterns (TMIN, TMAX, SNOW, PRECP) (weather of pothole occurrence date)

#### Lat and Lon from "the_geom" column
We have a feature of longtitudes and latitudes named as "the_geom", I'll feature engineer zip_codes and it will be our target variable.
For that I am first extracting the longitudes and latitudes from the_geom column using regex.
I am looping through the "the_geom" column and selecting the first pair of longitudes and latitudes in each row.

In [355]:
pattern = r'(-?\d+\.\d+)\s(-?\d+\.\d+)' #regex for extracting longtitude and latitude from the "the_geom" string

# Initialize empty lists to store longitude and latitude string
longitudes = []
latitudes = []

# Loop through each string in the list
for coordinates_string in df['the_geom']:
    # Find all matches of latitude and longitude using regular expression
    matches = re.findall(pattern, coordinates_string)
    # Extract the first pair of coordinates
    if matches:
        lon, lat = matches[0]
        longitudes.append(float(lon))
        latitudes.append(float(lat))

# Print the extracted coordinates
print("First pair of longitude and latitude:", longitudes[0] , latitudes[0])

First pair of longitude and latitude: -73.92993757815543 40.59714681302245


In [356]:
df['lon'] = longitudes
df['lat'] = latitudes

Then I am using the library "Geopy" and reversing the longitudes and latitudes to get the address and zip codes. The main challenge here was the computational limitations. we have 360,000 rows and for every pair of longitude and latitude, Geopy is making an HTTP get API call which was taking a long time to return our desired output.
To resolve that issue, I am using DOCKER on my local machine to improve the computation times. I have set up a "NominAtim" container for NYC geofabrik data. The command to create a container in docker is below.

"docker run -it   -e PBF_URL=https://download.geofabrik.de/north-america/us/new-york-latest.osm.pbf \
       -p 8080:8080   \
       --name nominatim2  mediagis/nominatim:4.4"

In [357]:
# from geopy.geocoders import Nominatim

# def get_address(latitude, longitude):

#     nominatim_url = '//localhost:8080/'

#     geolocator = Nominatim(user_agent="NYC_Potholes", domain = nominatim_url, scheme = 'http')
#     location = geolocator.reverse((str(latitude), str(longitude)), exactly_one=True)
#     # Extract zip code if available
#     zip_code = location.raw['address'].get('postcode')
#     building_number = location.raw['address'].get('building_number')
#     neighbour_hood = location.raw['address'].get('neighbour_hood')
#     street_name = location.raw['address'].get('street_name')
    
    
    
#     return zip_code, building_number, street_name, neighbour_hood

# zip_codes = []
# building_numbers = []
# neighbour_hoods = []
# street_names = []


# # Extract address information for each pair of coordinates
# for lat, lon in zip(latitudes, longitudes):
#     zip_code, building_number, street_name, neighbour_hood = get_address(lat, lon)
#     zip_codes.append(zip_code)
#     building_numbers.append(building_number)
#     street_names.append(street_name)
#     neighbour_hoods.append(neighbour_hood)

In [358]:
# df[zip_code] = zip_codes
# df[street_name] = street_names
# df.to_csv('updated_df')

I have intentionally commented on the above code for geopy, as it will take a long time even on the local machine, but a lot better than HTTP get request.
I am recreating a new dataset with our original features updated with zip_codes and street names (if we want to predict the street names in the future using NLP).
Importing the new data set for further feature engineering.

In [359]:
df = pd.read_csv('../Data/updated_df')

In [360]:
df.head()

,Unnamed: 0,the_geom,DefNum,InitBy,OFT,OnFaceName,OnPrimName,FrmPrimNam,ToPrimName,Boro,...,RptDate,RptClosed,Shape_Leng,index,lon,lat,zip_code,building_number,street_name,neighbour_hood
0,11,MULTILINESTRING ((-73.92993757815543 40.597146...,DB2010002001,CSC,343910312370314730,GERRITSEN AV,GERRITSEN AVENUE,ALLEN AVENUE,AVENUE W,B,...,2010-01-02,2010-01-05,82.707368,0,-73.929938,40.597147,11229,3206,Avenue W,Gerritsen Beach
1,12,MULTILINESTRING ((-73.9393340798436 40.6227256...,DB2010002004,CSC,336480314180354050,E 37 ST,EAST 37 STREET,AVENUE L,KINGS HIGHWAY,B,...,2010-01-02,2010-01-05,426.877885,1,-73.939334,40.622726,11210,1351,East 37th Street,NaN
2,13,MULTILINESTRING ((-73.96733404030275 40.581517...,DB2010003002,CSC,392230367730390730,W AV,WEST AVENUE,OCEAN PARKWAY,WEST 5 STREET,B,...,2010-01-03,2010-01-05,1071.289104,2,-73.967334,40.581518,11235,NaN,Ocean Parkway,West Brighton
3,14,MULTILINESTRING ((-73.95866587485429 40.712797...,DB2010003004,CSC,373650378130378230,ROEBLING ST,ROEBLING STREET,SOUTH 1 STREET,SOUTH 2 STREET,B,...,2010-01-03,2010-01-08,254.973102,3,-73.958666,40.712797,11211,227,South 1st Street,NaN
4,15,MULTILINESTRING ((-73.98873129769669 40.593001...,DB2010003005,CSC,307730315780347730,26 AV,26 AVENUE,BATH AVENUE,HARWAY AVENUE,B,...,2010-01-03,2010-01-05,729.958297,4,-73.988731,40.593002,11214,8782,26th Avenue,NaN


#### Days to fix a pothole
From the Rpt date and Close date of a pothole I have extracted the number of days it took to fix a pothole

In [361]:
# Days took to fix the pothole
df['RptDate'] = df['RptDate'].astype('datetime64[ns]') # converting to pandas date_time object
df['RptClosed'] = df['RptClosed'].astype('datetime64[ns]') # converting to pandas date_time object

date_diff = (df['RptClosed'] - df['RptDate']).dt.total_seconds() # subtracting RptDate and DateClosed columns
df['days_to_fix'] = date_diff / (60 * 60 * 24)  # Convert to days
df['days_to_fix'] = df['days_to_fix'].astype(float) 
df['days_to_fix']

0         3.0
1         3.0
2         2.0
3         5.0
4         2.0
         ... 
359889    1.0
359890    0.0
359891    0.0
359892    0.0
359893    0.0
Name: days_to_fix, Length: 359894, dtype: float64

#### Month of the pothole occurrence
I want to check the seasonal patterns for the occurrence of the potholes. For that, I am featuring engineering "month" and "year" columns from the "RptDate" column. First I have to convert it into pandas datetime format.
Then I calculated the number of days it took to fix a pothole.

In [362]:
df['month'] = df['RptDate'].dt.month_name()
df.shape

(359894, 23)

#### Year of pothole occurences

In [363]:
df['year'] = pd.to_datetime(df['RptDate'], format='%Y') #extracting year

#### Combining Weather Data with Pothole Data

I have tried multiple iterations of our model to learn from the pothole data but the results are not great. To introduce new features that might help our model, I am combining the NYC Central Park dataset with pothole data. This dataset contains Precipitation, snow, TMAX, TMIN
* combining on the dates of both datasets

In [364]:
weather_df = pd.read_csv("../Data/weather_nyc.csv")  # reading the weather CSV file

In [365]:
weather_df.drop(columns=['SNWD'], inplace=True)
weather_df.tail()

,DATE,PRCP,SNOW,TMIN,TMAX
56240,2022-12-25,0.0,0.0,14.0,28.0
56241,2022-12-26,0.0,0.0,18.0,29.0
56242,2022-12-27,0.0,0.0,29.0,35.0
56243,2022-12-28,0.0,0.0,33.0,47.0
56244,2022-12-29,0.0,0.0,40.0,51.0


In [366]:
null_counts = weather_df.isnull().sum()
null_counts 

DATE      0
PRCP      0
SNOW    163
TMIN      7
TMAX      7
dtype: int64

In [367]:
weather_df = weather_df[weather_df['DATE'] >= '2010-01-02'] # filtering the weather dataset where the potholes date ranges are starting from
weather_df

,DATE,PRCP,SNOW,TMIN,TMAX
51500,2010-01-02,0.02,0.2,17.0,34.0
51501,2010-01-03,0.00,0.0,17.0,22.0
51502,2010-01-04,0.00,0.0,19.0,30.0
51503,2010-01-05,0.00,0.0,20.0,30.0
51504,2010-01-06,0.00,0.0,26.0,34.0
...,...,...,...,...,...
56240,2022-12-25,0.00,0.0,14.0,28.0
56241,2022-12-26,0.00,0.0,18.0,29.0
56242,2022-12-27,0.00,0.0,29.0,35.0
56243,2022-12-28,0.00,0.0,33.0,47.0


There are few nulls in the weather data frame and I am filtering the weather data based on the time range of our potholes data.

We have a column named as "RPTDate" in potholes data, and a column named as "DATE" in weather data. I am combing both on the basis of the date. Since Weather data is until December of 2022 and potholes data is until December of 2023, Our data overall data will be limited to December 2022.

In [368]:
weather_df['DATE'] = weather_df['DATE'].astype('datetime64[ns]') 
df = pd.merge(df, weather_df, left_on='RptDate', right_on='DATE', how='inner')
df

,Unnamed: 0,the_geom,DefNum,InitBy,OFT,OnFaceName,OnPrimName,FrmPrimNam,ToPrimName,Boro,...,street_name,neighbour_hood,days_to_fix,month,year,DATE,PRCP,SNOW,TMIN,TMAX
0,11,MULTILINESTRING ((-73.92993757815543 40.597146...,DB2010002001,CSC,343910312370314730,GERRITSEN AV,GERRITSEN AVENUE,ALLEN AVENUE,AVENUE W,B,...,Avenue W,Gerritsen Beach,3.0,January,2010-01-02,2010-01-02,0.02,0.2,17.0,34.0
1,12,MULTILINESTRING ((-73.9393340798436 40.6227256...,DB2010002004,CSC,336480314180354050,E 37 ST,EAST 37 STREET,AVENUE L,KINGS HIGHWAY,B,...,East 37th Street,NaN,3.0,January,2010-01-02,2010-01-02,0.02,0.2,17.0,34.0
2,97384,MULTILINESTRING ((-73.95918180056893 40.771696...,DM2010002002,CSC,110210118490118510,3 AV,3 AVENUE,EAST 75 STREET,EAST 76 STREET,M,...,3rd Avenue,Manhattan Community Board 8,2.0,January,2010-01-02,2010-01-02,0.02,0.2,17.0,34.0
3,97390,MULTILINESTRING ((-73.95918180056893 40.771696...,DM2010003021,YRD,110210118490118510,3 AV,3 AVENUE,EAST 75 STREET,EAST 76 STREET,M,...,3rd Avenue,Manhattan Community Board 8,1.0,January,2010-01-02,2010-01-02,0.02,0.2,17.0,34.0
4,97391,MULTILINESTRING ((-73.95172338255492 40.773656...,DM2010003023,YRD,110010118610118630,1 AV,1 AVENUE,EAST 81 STREET,EAST 82 STREET,M,...,1st Avenue,Yorkville,1.0,January,2010-01-02,2010-01-02,0.02,0.2,17.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341660,275450,MULTILINESTRING ((-74.16758983553814 40.588953...,DS2012252004,CSC,544850524846544750,RICHMOND HILL RD,RICHMOND HILL ROAD,COUNTRY DRIVE WEST,RICHMOND AVENUE,S,...,Richmond Hill Road,New Springville,3.0,September,2012-09-08,2012-09-08,0.39,0.0,63.0,81.0
341661,322354,MULTILINESTRING ((-73.92245593511419 40.818045...,DX2012317004,YRD,226160221520252120,E 150 ST,EAST 150 STREET,COURTLANDT AVENUE,MORRIS AVENUE,X,...,East 150th Street,Bronx County,0.0,November,2012-11-11,2012-11-11,0.00,0.0,50.0,64.0
341662,322355,MULTILINESTRING ((-73.92288629831864 40.809105...,DX2012317005,YRD,225800215820278150,E 138 ST,EAST 138 STREET,BROWN PLACE,WILLIS AVENUE,X,...,East 138th Street,Mott Haven,0.0,November,2012-11-11,2012-11-11,0.00,0.0,50.0,64.0
341663,322356,MULTILINESTRING ((-73.86927391279683 40.856688...,DX2012317008,YRD,257720213820277150,PELHAM PY N,PELHAM PARKWAY,BOSTON ROAD,WHITE PLAINS ROAD,X,...,Bronx and Pelham Parkway,Bronx County,0.0,November,2012-11-11,2012-11-11,0.00,0.0,50.0,64.0


In [369]:
total_null_count = df.isnull().sum()
print(total_null_count)

Unnamed: 0              0
the_geom                0
DefNum                  0
InitBy                  0
OFT                     0
OnFaceName              0
OnPrimName              0
FrmPrimNam              0
ToPrimName              0
Boro                    0
Source                  0
RptDate                 0
RptClosed               0
Shape_Leng              0
index                   0
lon                     0
lat                     0
zip_code                0
building_number     80895
street_name          1326
neighbour_hood     105143
days_to_fix             0
month                   0
year                    0
DATE                    0
PRCP                    0
SNOW                    0
TMIN                    0
TMAX                    0
dtype: int64


One last sanity check for nulls in our data. we have large number of nulls in building number and neighbour_hood

Exporting the data frame with feature engineering and Data cleaning

In [370]:
df.to_csv('../Data/potholes_updated.csv')